Dataset pre-processed accordingly to EDA

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install transformers==4.28.0

!pip install -q datasets
!git config --global credential.helper store

!huggingface-cli login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.8 MB/s eta 0:00:00

    _|    _|  _

In [ ]:
import re
from collections import Counter
from datasets import load_dataset 

In [ ]:
# Functions for pre-processing

def filter_remove(example):
    return example[column_name] not in values_to_remove

def filter_remove2(example, column1, column2, values1, values2):
    return example[column1] not in values1 or (example[column1] in values1 and example[column2] not in values2)


def filter_keep(column_name, values_to_keep):
    def inner_filter(example):
        return example[column_name] in values_to_keep
    return inner_filter

def filter_keep2(column_name1, values_to_keep1, column_name2, values_to_keep2):
    def inner_filter(example):
        return (example[column_name1] in values_to_keep1) and (example[column_name2] in values_to_keep2)
    return inner_filter


def print_counts_in_descending_order(dataset, subcategory_value):
    filtered_dataset = dataset.filter(lambda example: example['subCategory'] == subcategory_value)
    article_type_counts = Counter(example['articleType'] for example in filtered_dataset)
    for article_type, count in article_type_counts.most_common():
        print(f"{article_type}: {count}")

In [ ]:
dataset = load_dataset("ashraq/fashion-product-images-small")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/44072 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/ashraq___parquet/ashraq--fashion-product-images-small-909c61a107795b31/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
train_ds=dataset["train"]

General preparation

In [ ]:
column_name = 'masterCategory'
values_to_remove = ['Personal Care', 'Free Items', 'Sporting Goods', 'Home']

dataset2 = train_ds.filter(filter_remove)

Filter:   0%|          | 0/44072 [00:00<?, ? examples/s]

In [ ]:
column_name = 'subCategory'
values_to_remove = ['Innerwear', 'Saree', 'Accessories', 'Cufflinks', 'Apparel Set', 'Gloves', 'Water Bottle', 'Umbrellas', 'Shoe Accessories', 'Sports Accessories', 'Perfumes', 'Loungewear and Nightwear']

dataset3 = dataset2.filter(filter_remove)

Filter:   0%|          | 0/41802 [00:00<?, ? examples/s]

In [ ]:
dataset4 = dataset3.map(lambda example: {'subCategory': 'Shoes' if example['subCategory'] in ['Sandal', 'Flip Flops'] else example['subCategory']})
dataset4 = dataset4.map(lambda example: {'subCategory': 'Scarves' if example['subCategory'] in ['Stoles', 'Mufflers'] else example['subCategory']})

Map:   0%|          | 0/38721 [00:00<?, ? examples/s]

Map:   0%|          | 0/38721 [00:00<?, ? examples/s]

In [ ]:
dataset5 = dataset4.filter(lambda example: filter_remove2(example, 'subCategory', 'articleType', ['Topwear'], ['Kurtas', 'Kurtis', 'Dupatta', 'Rompers', 'Shrug', 'Nehru Jackets', 'Lehenga Choli', 'Belts', 'Dresses', 'Tunics']))
dataset5 = dataset5.filter(lambda example: filter_remove2(example, 'subCategory', 'articleType', ['Bags'], ['Wallets']))
dataset5 = dataset5.filter(lambda example: filter_remove2(example, 'subCategory', 'articleType', ['Bottomwear'], ['Stockings', 'Tracksuits', 'Swimwear', 'Tights', 'Salwar and Dupatta']))
dataset5 = dataset5.filter(lambda example: filter_remove2(example, 'subCategory', 'articleType', ['Belts'], ['Tshirts']))
dataset5 = dataset5.filter(lambda example: filter_remove2(example, 'subCategory', 'articleType', ['Socks'], ['Booties']))
dataset5 = dataset5.filter(lambda example: filter_remove2(example, 'subCategory', 'articleType', ['Dress'], ['Jumpsuit']))

Filter:   0%|          | 0/38721 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36266 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36264 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36174 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36173 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36161 [00:00<?, ? examples/s]

Some checks

In [ ]:
print_counts_in_descending_order(dataset5, 'Topwear')

Filter:   0%|          | 0/36145 [00:00<?, ? examples/s]

Tshirts: 7064
Shirts: 3212
Tops: 1762
Sweatshirts: 285
Sweaters: 277
Jackets: 258
Suspenders: 40
Waistcoat: 15
Blazers: 8
Rain Jacket: 7


In [ ]:
print_counts_in_descending_order(dataset5, 'Bags')

Filter:   0%|          | 0/36145 [00:00<?, ? examples/s]

Handbags: 1757
Backpacks: 722
Clutches: 287
Duffel Bag: 88
Laptop Bag: 81
Mobile Pouch: 47
Messenger Bag: 34
Waist Pouch: 17
Rucksacks: 11
Trolley Bag: 3
Tablet Sleeve: 3
Travel Accessory: 1


In [ ]:
print_counts_in_descending_order(dataset5, 'Belts')

Filter:   0%|          | 0/36145 [00:00<?, ? examples/s]

Belts: 810


In [ ]:
print_counts_in_descending_order(dataset5, 'Socks')

Filter:   0%|          | 0/36145 [00:00<?, ? examples/s]

Socks: 686


In [ ]:
print_counts_in_descending_order(dataset5, 'Dress')

Filter:   0%|          | 0/36145 [00:00<?, ? examples/s]

Dresses: 462


In [ ]:
dataset6 = dataset5.select(range(len(dataset5)))

holder1 = [text[text.find("Men"):] if "Men" in text else text for text in dataset5['productDisplayName']]
holder2 = [text[text.find("Men's"):] if "Men's" in text else text for text in holder1]
holder3 = [text[text.find("Women"):] if "Women" in text else text for text in holder2]
holder4 = [text[text.find("Women's"):] if "Women's" in text else text for text in holder3]
holder5 = [text[text.find("Boys"):] if "Boys" in text else text for text in holder4]
holder6 = [text[text.find("Boy's"):] if "Boy's" in text else text for text in holder5]
holder7 = [text[text.find("Girls"):] if "Girls" in text else text for text in holder6]
holder8 = [text[text.find("Girl's"):] if "Girl's" in text else text for text in holder7]

dataset6 = dataset6.remove_columns(['id', 'gender', 'masterCategory', 'articleType', 'baseColour', 'season', 'year', 'usage'])
dataset6 = dataset6.add_column('text', holder8)
dataset6

Flattening the indices:   0%|          | 0/36145 [00:00<?, ? examples/s]

Dataset({
    features: ['subCategory', 'productDisplayName', 'image', 'text'],
    num_rows: 36145
})

In [ ]:
for i in range(5):
    print(f"Row {i+1}:")
    print(f"  subCategory: {dataset6['subCategory'][i]}")
    print(f"  productDisplayName: {dataset6['productDisplayName'][i]}")
    print(f"  image: {dataset6['image'][i]}")
    print(f"  text: {dataset6['text'][i]}")
    print()

Row 1:
  subCategory: Topwear
  productDisplayName: Turtle Check Men Navy Blue Shirt
  image: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F328C8EAA70>
  text: Men Navy Blue Shirt

Row 2:
  subCategory: Bottomwear
  productDisplayName: Peter England Men Party Blue Jeans
  image: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F32841702E0>
  text: Men Party Blue Jeans

Row 3:
  subCategory: Watches
  productDisplayName: Titan Women Silver Watch
  image: <PIL.JpegImagePlugin.JpegImageFile image mode=L size=60x80 at 0x7F328416F5E0>
  text: Women Silver Watch

Row 4:
  subCategory: Bottomwear
  productDisplayName: Manchester United Men Solid Black Track Pants
  image: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F328416F520>
  text: Men Solid Black Track Pants

Row 5:
  subCategory: Topwear
  productDisplayName: Puma Men Grey T-shirt
  image: <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=60x80 at 0x7F32923DF9D0>
  text

In [ ]:
dataset7 = dataset6.remove_columns(['productDisplayName'])
dataset7

Dataset({
    features: ['subCategory', 'image', 'text'],
    num_rows: 36145
})

In [ ]:
!huggingface-cli repo create preproc-fashion-products --type dataset

git version 2.25.1
git-lfs/2.9.2 (GitHub; linux amd64; go 1.13.5)

You are about to create datasets/agestau/preproc-fashion-products
Proceed? [Y/n] Y

Your repo now lives at:
  https://huggingface.co/datasets/agestau/preproc-fashion-products

You can clone it locally with the command below, and commit/push as usual.

  git clone https://huggingface.co/datasets/agestau/preproc-fashion-products



In [ ]:
dataset7.push_to_hub("agestau/preproc-fashion-products")

Map:   0%|          | 0/36145 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/37 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]